## Tensorflow Abstractions

In [2]:
from sklearn.datasets import load_wine

In [3]:
wine_data = load_wine()

In [4]:
type(wine_data)

sklearn.utils.Bunch

In [5]:
wine_data.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names'])

In [6]:
print(wine_data["DESCR"])

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            1

In [7]:
feat_data = wine_data["data"]
labels = wine_data["target"]

In [8]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(feat_data, labels, test_size=0.3, random_state=101)

In [12]:
from sklearn.preprocessing import MinMaxScaler

In [26]:
scaler = MinMaxScaler()
scaled_x_train = scaler.fit_transform(X_train)
scaled_x_test = scaler.transform(X_test)

In [27]:
import tensorflow as tf
from tensorflow import estimator

In [28]:
X_train.shape

(124, 13)

In [29]:
feat_cols = [tf.feature_column.numeric_column("x", shape=[13])]

In [30]:
deep_model = estimator.DNNClassifier(hidden_units=[13,13,13], feature_columns=feat_cols, n_classes=3,
                                     optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.01))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\marco\\AppData\\Local\\Temp\\tmpmuugjr33', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001F6F6196448>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [31]:
input_fn = estimator.inputs.numpy_input_fn(x={"x": scaled_x_train}, y=y_train, shuffle=True, batch_size=10, num_epochs=5)

In [32]:
deep_model.train(input_fn=input_fn, steps = 500)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\marco\AppData\Local\Temp\tmpmuugjr33\model.ckpt.
INFO:tensorflow:loss = 10.584951, step = 1
INFO:tensorflow:Saving checkpoints for 62 into C:\Users\marco\AppData\Local\Temp\tmpmuugjr33\model.ckpt.
INFO:tensorflow:Loss for final step: 5.249896.


In [33]:
input_fn_eval = estimator.inputs.numpy_input_fn(x={"x":scaled_x_test}, shuffle=False)

In [34]:
preds = list(deep_model.predict(input_fn=input_fn_eval))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\marco\AppData\Local\Temp\tmpmuugjr33\model.ckpt-62
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [35]:
predictions = [p["class_ids"][0] for p in preds]

In [36]:
from sklearn.metrics import confusion_matrix, classification_report

In [37]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.90      1.00      0.95        19
           1       1.00      0.91      0.95        22
           2       1.00      1.00      1.00        13

    accuracy                           0.96        54
   macro avg       0.97      0.97      0.97        54
weighted avg       0.97      0.96      0.96        54

